# Filtro de Kalman

In [273]:
import numpy as np

In [274]:
def determinarDF(x, y, theta, Sx, Sy):
    dF = 0
    x = x[0][0]
    y = y[0][0]
    theta = theta[0]
    print(x, y, theta)
    if theta >= 0 and theta < np.pi / 2:
        dF = (Sy - y) / np.abs(np.cos(theta)) 
    elif theta >= np.pi / 2 and theta < np.pi:
        dF = (Sx - x) / np.abs(np.sin(theta))
    elif theta >= np.pi and theta < 3*np.pi/2:
        dF = y / np.abs(np.cos(theta)) 
    elif theta >= 3 * np.pi / 2 and theta < 2 * np.pi:
        dF = x / np.abs(np.sin(theta))        
    return dF

def determinarDR(x, y, theta, Sx, Sy):
    dR = 0
    x = x[0][0]
    y = y[0][0]
    theta = theta[0]
    if theta >= 0 and theta < np.pi / 2:
        dR = (Sx - x) / np.abs(np.cos(theta)) 
    elif theta >= np.pi / 2 and theta < np.pi:
        dR = (Sx - x) / np.abs(np.sin(theta))
    elif theta >= np.pi and theta < 3*np.pi/2:
        dR = x / np.abs(np.cos(theta)) 
    elif theta >= 3 * np.pi / 2 and theta < 2 * np.pi:
        dR = (Sy - y) / np.abs(np.sin(theta))        
    return dR

def determinarDL(x, y, theta, Sx, Sy):
    dL = 0
    x = x[0][0]
    y = y[0][0]
    theta = theta[0]
    if theta >= 0 and theta < np.pi / 2:
        dL = x / np.abs(np.cos(theta)) 
    elif theta >= np.pi / 2 and theta < np.pi:
        dL = (Sy - y) / np.abs(np.sin(theta))
    elif theta >= np.pi and theta < 3*np.pi/2:
        dL = (Sx - x) / np.abs(np.cos(theta)) 
    elif theta >= 3 * np.pi / 2 and theta < 2 * np.pi:
        dL = y / np.abs(np.sin(theta))        
    return dL

    
def determinarMatrizH(x, y, theta, Sx, Sy):
    x = x[0][0]
    y = y[0][0]
    theta = theta[0]
    if theta >= 0 and theta < np.pi / 2:
        H = np.array([[0, -1/np.abs(np.cos(theta)), 0, 0, (((Sy - y)*np.tan(theta)) / np.abs(np.cos(theta))), 0, 0],
                       [-1/np.abs(np.cos(theta)), 0, 0, 0,(((Sx - x)*np.tan(theta)) / np.abs(np.cos(theta))), 0, 0],
                       [-1/np.abs(np.cos(theta)), 0, 0, 0, (((x)*np.tan(theta)) / np.abs(np.cos(theta))), 0, 0],
                       [0, 0, 0, 0, 0],
                       [0, 0, 0, 0, 0]])
        print(H.shape)
    elif theta >= np.pi / 2 and theta < np.pi:
        H = np.array([[-1/np.abs(np.sin(theta)), 0, 0, 0, ((-(Sx - x)*np.cos(theta)) / (np.sin(theta) * np.abs(np.sin(theta)))), 0, 0],
                       [0, 1/np.abs(np.sin(theta)),  0, 0, ((-(y)*np.cos(theta)) / (np.sin(theta) * np.abs(np.sin(theta)))), 0, 0],
                       [0, -1/np.abs(np.sin(theta)), 0, 0, ((-(Sy - y)*np.cos(theta)) / (np.sin(theta) * np.abs(np.sin(theta)))), 0, 0],
                       [0, 0, 0, 0, 0],
                       [0, 0, 0, 0, 0]])
        print(H.shape)
    elif theta >= np.pi and theta < 3*np.pi/2:
        H = np.array([[0, 1/np.abs(np.cos(theta)), 0, 0, (((y)*np.tan(theta)) / np.abs(np.cos(theta))), 0, 0],
                       [1/np.abs(np.cos(theta)), 0, 0, 0, (((x)*np.tan(theta)) / np.abs(np.cos(theta))), 0, 0],
                       [-1/np.abs(np.cos(theta)), 0, 0, 0, (((Sx - x)*np.tan(theta)) / np.abs(np.cos(theta))), 0, 0],
                       [0, 0, 0, 0, 0],
                       [0, 0, 0, 0, 0]])
        print(H.shape)
    elif theta >= 3 * np.pi / 2 and theta < 2 * np.pi:
        H = np.array([[1/np.abs(np.sin(theta)), 0,   0, 0, ((-(x)*np.cos(theta)) / (np.sin(theta) * np.abs(np.sin(theta)))), 0, 0],
                       [0, -1/np.abs(np.sin(theta)),  0, 0, ((-(Sy - y)*np.cos(theta)) / (np.sin(theta) * np.abs(np.sin(theta)))), 0, 0],
                       [0, -1/np.abs(np.sin(theta)), 0, 0, ((-(y)*np.cos(theta)) / (np.sin(theta) * np.abs(np.sin(theta)))), 0, 0],
                       [0, 0, 0, 0, 0],
                       [0, 0, 0, 0, 0]])
        print(H.shape)
        

    print(H.shape)
    return H

In [275]:
def filtroDeKalman(posicao_anterior, deltaT, u_k, P, distancias_lidas):
    posicao_anterior = (np.array([posicao_anterior])).transpose()
    distancias = np.array([distancias_lidas])
    x_kmenos1, y_kmenos1, vx_kmenos1, vy_kmenos1, theta_kmenos1 = posicao_anterior[0], posicao_anterior[1], posicao_anterior[2], posicao_anterior[3], posicao_anterior[4] 
    aceleracao_medida, velocidade_angular = u_k[0], u_k[1]
    alpha = theta_kmenos1 + (deltaT*velocidade_angular) 
    deltaX = (1/2 * (deltaT ** 2) * aceleracao_medida  + vx_kmenos1*deltaT) * np.cos(alpha)
    deltaY = (1/2 * (deltaT ** 2) * aceleracao_medida + vy_kmenos1*deltaT) * np.sin(alpha)
    deltaVx = aceleracao_medida * deltaT * np.cos(alpha)
    deltaVy = aceleracao_medida * deltaT * np.sin(alpha)
    deltaTheta = deltaT * velocidade_angular
    posicao_predita = posicao_anterior + np.array([[deltaX],
                                                    [deltaY],
                                                    [deltaVx],
                                                    [deltaVy],
                                                    [deltaTheta]])
    F_p = np.array([[1, 0, deltaT * np.cos(alpha), 0, ((-1/2 * (deltaT ** 2) * aceleracao_medida - vx_kmenos1 * deltaT) * np.sin(alpha))],
                     [0, 1, 0, deltaT * np.sin(alpha), ((1/2 * (deltaT ** 2) * aceleracao_medida + vy_kmenos1 * deltaT) * np.cos(alpha))],
                     [0, 0, 1, 0, (-1)*(aceleracao_medida * deltaT * np.sin(alpha))],
                     [0, 0, 0, 1, (aceleracao_medida * deltaT * np.cos(alpha))],
                     [0, 0, 0, 0, 1]])
    
    F_u = np.array([[1/2 * (deltaT ** 2) * np.cos(alpha), 
                        ((-1/2 * (deltaT ** 3) * aceleracao_medida) - (vx_kmenos1 * (deltaT ** 2))) * np.sin(alpha)],
                     [1/2 * (deltaT ** 2) * np.sin(alpha),
                         ((1/2 * (deltaT ** 3) * aceleracao_medida) + (vy_kmenos1 * (deltaT ** 2))) * np.cos(alpha)],
                     [deltaT * np.cos(alpha), (-1)*(aceleracao_medida * deltaT * np.sin(alpha))],
                     [deltaT * np.sin(alpha), (aceleracao_medida * deltaT * np.cos(alpha))],
                     [0, deltaT]])
    
    Q = np.array([[1, 0],
                   [0, 1]])
    print(F_u.shape)
    P_predito = (F_p.dot(P)).dot(F_p.transpose()) + (F_u.dot(Q)).dot(F_u.transpose())
    Sx, Sy = 1, 1
    hDF = determinarDF(posicao_predita[0], posicao_predita[1], posicao_predita[4], Sx, Sy)
    hDR = determinarDR(posicao_predita[0], posicao_predita[1], posicao_predita[4], Sx, Sy)
    hDL = determinarDL(posicao_predita[0], posicao_predita[1], posicao_predita[4], Sx, Sy)
    funcaoH = [hDF, hDR, hDL]
    Vk = distancias - np.array([funcaoH])
    H = determinarMatrizH(posicao_predita[0], posicao_predita[1], posicao_predita[4], Sx, Sy)
    Rk = np.array([[0.01, 0, 0, 0, 0],
                    [0, 0.01, 0, 0, 0],
                    [0, 0, 0.01, 0, 0],
                    [0, 0, 0, 0.01, 0],
                    [0, 0, 0, 0, 0.01]])
    print(P_predito.shape)
    Sk = (H.dot(P_predito)).dot(H.transpose()) + Rk
    Kk = (P_predito.dot(H.transpose())).dot(np.linalg.inv(Sk)) 
    posicao_atual = posicao_predita + (Kk.dot(Vk))
    P_atual = P_predito + ((Kk.dot(Sk)).dot(Kk.transpose()))
    return posicao_atual, P_atual

In [276]:
A = np.array([[6, 1, 1, 3], 
              [4, -2, 5, 1], 
              [2, 8, 7, 6], 
              [3, 1, 9, 7]]) 
print(A.shape)
print(np.linalg.inv(A))

(4, 4)
[[ 0.13368984  0.10695187  0.02139037 -0.09090909]
 [-0.00229183  0.02673797  0.14820474 -0.12987013]
 [-0.12987013  0.18181818  0.06493506 -0.02597403]
 [ 0.11000764 -0.28342246 -0.11382735  0.23376623]]


In [277]:
P = np.matrix([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]])
filtroDeKalman([0, 0, 0, 0, 0], 0.1, [0.1, 0.3], P, [0.2, 0.2, 0.2])

# dF (x, y, theta) = {
#     * 0 <= theta < pi/2:      |Yp - Sy| / |cos(theta)|
#     * pi/2 <= theta < pi:     |Xp - Sx| / |sin(theta)|
#     * pi <= theta < 3*pi/2:   Yp / |cos(theta)|
#     * 3*pi/2 <= theta < 2*pi: Xp / |sin(theta)|  
# }

# dR (x, y, theta) = {
#     * 0 <= theta < pi/2:      |Sx - Xp| / |cos(theta)
#     * pi/2 <= theta < pi:     Yp / |sin(theta)|
#     * pi <= theta < 3*pi/2:   Xp / |cos(theta)|
#     * 3*pi/2 <= theta < 2*pi: |Sy - Yp| / |sin(theta)| 
# }

# dL (x, y, theta) = {
#     * 0 <= theta < pi/2:      Xp / |cos(theta)
#     * pi/2 <= theta < pi:     |Sy - Yp| / |sin(theta)|
#     * pi <= theta < 3*pi/2:   |Sx - Xp| / |cos(theta)|
#     * 3*pi/2 <= theta < 2*pi: Yp / |sin(theta)| 
# }

(5, 2)
0.0004997750168744939 1.4997750101247834e-05 0.03
(5,)
(5,)
(5, 5)


<ipython-input-275-9c987c92d63a>:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  posicao_predita = posicao_anterior + np.array([[deltaX],
<ipython-input-275-9c987c92d63a>:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  F_p = np.array([[1, 0, deltaT * np.cos(alpha), 0, ((-1/2 * (deltaT ** 2) * aceleracao_medida - vx_kmenos1 * deltaT) * np.sin(alpha))],
<ipython-input-275-9c987c92d63a>:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or sh

ValueError: operands could not be broadcast together with shapes (7,) (5,) 